In [24]:
import duckdb, pandas as pd, numpy as np

file_train_cls = 'data/NMAD_with_embeddings_cls.parquet'
con = duckdb.connect(file_train_cls)

# забираємо все потрібне під FABDEM
df = con.execute("""
  SELECT
    -- групування / просторові поля
    rgt, track, spot, x, y,
    -- таргет FABDEM
    cls_nmad_fab, nmad_fab,
    h_fab_dem,
    fab_dem_slope, fab_dem_twi, fab_dem_2000, fab_dem_stream,

    -- категоріальні LULC/geomorphon/landform
    lulc_class, lulc_name, fab_dem_geomorphon, fab_dem_landform,
    -- ембедінги (64 колонок або emb_all)
    emb_001, emb_002, emb_003, emb_004, emb_005, emb_006, emb_007, emb_008,
    emb_009, emb_010, emb_011, emb_012, emb_013, emb_014, emb_015, emb_016,
    emb_017, emb_018, emb_019, emb_020, emb_021, emb_022, emb_023, emb_024,
    emb_025, emb_026, emb_027, emb_028, emb_029, emb_030, emb_031, emb_032,
    emb_033, emb_034, emb_035, emb_036, emb_037, emb_038, emb_039, emb_040,
    emb_041, emb_042, emb_043, emb_044, emb_045, emb_046, emb_047, emb_048,
    emb_049, emb_050, emb_051, emb_052, emb_053, emb_054, emb_055, emb_056,
    emb_057, emb_058, emb_059, emb_060, emb_061, emb_062, emb_063, emb_064,
    -- або emb_all якщо зберігав як масив/список
  FROM NMAD_with_embeddings_cls
""").fetchdf()

# Таргет
df["cls_nmad_fab"] = df["cls_nmad_fab"].astype(int)

# Група для spatial split
df["group_id"] = (df["rgt"].astype(int).astype(str) + "_" +
                  df["track"].astype(int).astype(str))


In [25]:
# рахуємо розмір груп
gsize = df.groupby("group_id").size().rename("grp_n")  # <-- унікальна назва!

# приєднуємо розмір груп до df (без конфлікту імен)
df = df.join(gsize, on="group_id")

MIN_TEST_N = 3000
df["is_big_group"] = df["grp_n"] >= MIN_TEST_N


In [26]:
num_cols = [
    "x", "y","h_fab_dem",
    "fab_dem_slope","fab_dem_twi","fab_dem_2000","fab_dem_stream"
]
cat_cols = ["lulc_class","lulc_name","fab_dem_geomorphon","fab_dem_landform"]

# ембедінги (якщо є всі 64 канали окремими колонками)
emb_cols = [f"emb_{i:03d}" for i in range(1,65) if f"emb_{i:03d}" in df.columns]


In [27]:
def collapse_rare(df, col, min_frac=0.01):
    vc = df[col].value_counts(normalize=True)
    rare = vc[vc < min_frac].index
    df[col] = df[col].where(~df[col].isin(rare), "__OTHER__")
    return df

for c in cat_cols:
    if c in df.columns:
        df[c] = df[c].astype(str)
        df = collapse_rare(df, c, 0.01)


In [28]:
from sklearn.model_selection import StratifiedGroupKFold

y = df["cls_nmad_fab"].astype(int)
groups = df["group_id"]
is_big = df["is_big_group"].values

sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

folds = []
big_idx = np.where(is_big)[0]
y_big   = y.iloc[big_idx]
grp_big = groups.iloc[big_idx]

for fold, (tr_big, te_big) in enumerate(sgkf.split(big_idx, y_big, groups=grp_big), 1):
    tr_idx = big_idx[tr_big]
    te_idx = big_idx[te_big]
    # усі "малі" групи — тільки в train
    tr_idx = np.concatenate([tr_idx, np.where(~is_big)[0]])
    folds.append((np.sort(tr_idx), np.sort(te_idx)))

    print(
        f"Fold {fold}: "
        f"train groups = {df.iloc[tr_idx]['group_id'].nunique()} | "
        f"test groups = {df.iloc[te_idx]['group_id'].nunique()}"
    )

# Перевірки
for k, (tr, te) in enumerate(folds, 1):
    assert set(df.iloc[tr]["group_id"]).isdisjoint(set(df.iloc[te]["group_id"]))
    print(f"Fold {k} class dist train:",
          y.iloc[tr].value_counts(normalize=True).sort_index().round(3).to_dict(),
          "| test:",
          y.iloc[te].value_counts(normalize=True).sort_index().round(3).to_dict())


Fold 1: train groups = 8 | test groups = 3
Fold 2: train groups = 8 | test groups = 3
Fold 3: train groups = 9 | test groups = 2
Fold 4: train groups = 10 | test groups = 1
Fold 5: train groups = 10 | test groups = 1
Fold 1 class dist train: {0: 0.332, 1: 0.329, 2: 0.34} | test: {0: 0.326, 1: 0.332, 2: 0.342}
Fold 2 class dist train: {0: 0.326, 1: 0.332, 2: 0.343} | test: {0: 0.338, 1: 0.326, 2: 0.337}
Fold 3 class dist train: {0: 0.331, 1: 0.333, 2: 0.336} | test: {0: 0.319, 1: 0.308, 2: 0.372}
Fold 4 class dist train: {0: 0.33, 1: 0.326, 2: 0.344} | test: {0: 0.328, 1: 0.359, 2: 0.313}
Fold 5 class dist train: {0: 0.33, 1: 0.33, 2: 0.34} | test: {0: 0.33, 1: 0.327, 2: 0.343}


In [38]:
from sklearn.decomposition import PCA

# ембедінги (64-колонки)
Z = df[emb_cols].to_numpy(dtype="float32")

# PCA з 32 компонентами
pca = PCA(n_components=24, random_state=42)
pca.fit(Z)

# explained variance ratio (по кожному компоненту)
evr = pca.explained_variance_ratio_

# кумулятивна сума — скільки % дисперсії накопичилось
print("EVR (по кожній компоненті):", evr)
print("EVR cumulative (24):", evr.cumsum()[-1])


EVR (по кожній компоненті): [0.34533104 0.13338497 0.10327472 0.09242915 0.0495403  0.0450718
 0.03297483 0.02686134 0.02141951 0.01786984 0.01509036 0.01256596
 0.01103213 0.00947172 0.00858145 0.00828346 0.00690836 0.00612718
 0.00570824 0.00465606 0.00392586 0.00368238 0.0031934  0.00287302]
EVR cumulative (24): 0.970257


In [39]:
import os, json
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, balanced_accuracy_score, confusion_matrix, classification_report
from catboost import CatBoostClassifier, Pool
from sklearn.decomposition import PCA
import joblib

# --- налаштування ---
SAVE_DIR = "artifacts_fabdem_cv"
os.makedirs(SAVE_DIR, exist_ok=True)
REDUCED_DIM = 48  # скільки компонент залишати з 64-ембедінгів


# --- підготовка PCA об'єкта (фітитимемо на train усередині 1-го фолду) ---
pca = PCA(n_components=REDUCED_DIM, random_state=42) if len(emb_cols) > 0 else None


all_fold_metrics = []
best_model = None
best_macro_f1 = -1.0
best_fold_id = None

# для збереження усіх тестових прогнозів по фолдах
test_preds_concat = []

for k, (tr, te) in enumerate(folds, 1):
    print(f"\n===== FOLD {k} =====")
    y_tr, y_te = y.iloc[tr], y.iloc[te]

    # числові/категоріальні
    Xn_tr = df.loc[tr, num_cols].copy()
    Xn_te = df.loc[te, num_cols].copy()
    Xc_tr = df.loc[tr, cat_cols].astype(str).copy()
    Xc_te = df.loc[te, cat_cols].astype(str).copy()

    # ембедінги + PCA (fit на train у першому фолді, далі тільки transform)
    if len(emb_cols) > 0:
        Z_tr = df.loc[tr, emb_cols].to_numpy(dtype="float32")
        Z_te = df.loc[te, emb_cols].to_numpy(dtype="float32")
        if k == 1:
            pca.fit(Z_tr)
        Z_tr = pca.transform(Z_tr)
        Z_te = pca.transform(Z_te)
        Ztr_df = pd.DataFrame(Z_tr, index=Xn_tr.index, columns=[f"pca_emb_{i:02d}" for i in range(REDUCED_DIM)])
        Zte_df = pd.DataFrame(Z_te, index=Xn_te.index, columns=[f"pca_emb_{i:02d}" for i in range(REDUCED_DIM)])
        Xn_tr = pd.concat([Xn_tr, Ztr_df], axis=1)
        Xn_te = pd.concat([Xn_te, Zte_df], axis=1)

    # фінальна матриця фіч
    X_tr = pd.concat([Xn_tr, Xc_tr], axis=1)
    X_te = pd.concat([Xn_te, Xc_te], axis=1)
    cat_idx = [X_tr.columns.get_loc(c) for c in Xc_tr.columns]

    # пули
    train_pool = Pool(X_tr, y_tr, cat_features=cat_idx)
    test_pool  = Pool(X_te, y_te, cat_features=cat_idx)

    # модель
    model = CatBoostClassifier(
        loss_function="MultiClass",
        iterations=3000, learning_rate=0.05, depth=8,
        l2_leaf_reg=8, auto_class_weights="Balanced",
        random_state=42, task_type="GPU",
        od_type="Iter", od_wait=100, verbose=200
    )
    model.fit(train_pool, eval_set=test_pool, use_best_model=True)

    # прогнози
    y_pred = model.predict(test_pool).astype(int).ravel()
    y_proba = model.predict_proba(test_pool)  # shape: (N, 3)
    macro_f1 = f1_score(y_te, y_pred, average="macro")
    bal_acc  = balanced_accuracy_score(y_te, y_pred)

    # per-class F1
    f1_per_class = f1_score(y_te, y_pred, average=None)  # порядок класів 0,1,2
    cm = confusion_matrix(y_te, y_pred, labels=[0,1,2])

    print(f"FOLD {k}  macro-F1={macro_f1:.3f}  balanced-acc={bal_acc:.3f}")
    print("Per-class F1:", {c: round(v,3) for c, v in zip([0,1,2], f1_per_class)})
    print("Confusion matrix (rows=true, cols=pred):\n", cm)

    # зберегти метрики фолду
    all_fold_metrics.append({
        "fold": k,
        "macro_f1": float(macro_f1),
        "balanced_acc": float(bal_acc),
        "f1_class_0": float(f1_per_class[0]),
        "f1_class_1": float(f1_per_class[1]),
        "f1_class_2": float(f1_per_class[2]),
        "cm_00": int(cm[0,0]), "cm_01": int(cm[0,1]), "cm_02": int(cm[0,2]),
        "cm_10": int(cm[1,0]), "cm_11": int(cm[1,1]), "cm_12": int(cm[1,2]),
        "cm_20": int(cm[2,0]), "cm_21": int(cm[2,1]), "cm_22": int(cm[2,2]),
    })

    # трекінг найкращої моделі
    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        best_model = model
        best_fold_id = k

    # збережемо тестові прогнози цього фолду з координатами — для карти та аналізу
    out_te = df.loc[te, ["x","y","rgt","track","spot"]].copy()
    out_te["y_true"] = y_te.values
    out_te["y_pred"] = y_pred
    out_te["prob_low"]  = y_proba[:,0]
    out_te["prob_mid"]  = y_proba[:,1]
    out_te["prob_high"] = y_proba[:,2]
    out_te["fold"] = k
    test_preds_concat.append(out_te)

# --- підсумки по фолдах ---
metrics_df = pd.DataFrame(all_fold_metrics)
metrics_df.to_csv(os.path.join(SAVE_DIR, "cv_metrics.csv"), index=False)

print("\n===== CV SUMMARY =====")
for col in ["macro_f1", "balanced_acc", "f1_class_0", "f1_class_1", "f1_class_2"]:
    mu, sd = metrics_df[col].mean(), metrics_df[col].std()
    print(f"{col}: {mu:.3f} ± {sd:.3f}")

print(f"\nBest fold = {best_fold_id}  (macro-F1={best_macro_f1:.3f})")

# збереження найкращої моделі та (за потреби) PCA + маніфест колонок
best_model.save_model(os.path.join(SAVE_DIR, "catboost_fabdem_best.cbm"))
if pca is not None:
    joblib.dump(pca, os.path.join(SAVE_DIR, "pca_embeddings.pkl"))

columns_manifest = {
    "numeric": list(X_tr.select_dtypes(include=[np.number]).columns),
    "categorical": list(Xc_tr.columns),
    "target": "cls_nmad_fab",
    "pca_dim": REDUCED_DIM if pca is not None else 0
}
json.dump(columns_manifest, open(os.path.join(SAVE_DIR, "columns_manifest.json"), "w"), ensure_ascii=False, indent=2)

# усі тестові прогнози разом (для швидкої перевірки на карті)
test_preds_df = pd.concat(test_preds_concat, axis=0).reset_index(drop=True)
test_preds_df.to_parquet(os.path.join(SAVE_DIR, "cv_test_predictions.parquet"))
print("\nSaved:")
print(" - CV metrics ->", os.path.join(SAVE_DIR, "cv_metrics.csv"))
print(" - Best model ->", os.path.join(SAVE_DIR, "catboost_fabdem_best.cbm"))
print(" - Columns manifest ->", os.path.join(SAVE_DIR, "columns_manifest.json"))
print(" - PCA (optional) ->", os.path.join(SAVE_DIR, "pca_embeddings.pkl"))
print(" - Test preds ->", os.path.join(SAVE_DIR, "cv_test_predictions.parquet"))



===== FOLD 1 =====
0:	learn: 1.0874247	test: 1.0902800	best: 1.0902800 (0)	total: 44.5ms	remaining: 2m 13s
200:	learn: 0.8453768	test: 0.9756905	best: 0.9755776 (197)	total: 7.46s	remaining: 1m 43s
400:	learn: 0.7888125	test: 0.9741398	best: 0.9736605 (329)	total: 14.9s	remaining: 1m 36s
bestTest = 0.97366049
bestIteration = 329
Shrink model to first 330 iterations.
FOLD 1  macro-F1=0.498  balanced-acc=0.505
Per-class F1: {0: np.float64(0.458), 1: np.float64(0.415), 2: np.float64(0.622)}
Confusion matrix (rows=true, cols=pred):
 [[109723  51772  62554]
 [ 93994  79947  54476]
 [ 51113  25192 158858]]

===== FOLD 2 =====
0:	learn: 1.0870629	test: 1.0889423	best: 1.0889423 (0)	total: 38.4ms	remaining: 1m 55s
200:	learn: 0.8348082	test: 0.9689741	best: 0.9689741 (200)	total: 7.44s	remaining: 1m 43s
bestTest = 0.9683644348
bestIteration = 279
Shrink model to first 280 iterations.
FOLD 2  macro-F1=0.503  balanced-acc=0.514
Per-class F1: {0: np.float64(0.461), 1: np.float64(0.4), 2: np.floa

In [40]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap

# 1) читаємо parquet
df = pd.read_parquet("artifacts_fabdem_cv/cv_test_predictions.parquet")

# 2) прапор помилки (плутає клас)
df["err"] = (df["y_true"] != df["y_pred"]).astype(int)

# 3) GeoDataFrame у метрах (EPSG:3857)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["x"], df["y"]), crs="EPSG:4326")

# 4) для folium треба широту/довготу → трансформуємо в WGS84
gdf_wgs = gdf.to_crs(4326)

# 5) беремо тільки помилки (або зважуємо вагою err)
pts = gdf_wgs.loc[gdf_wgs["err"] == 1, ["geometry"]].copy()
pts["lat"] = pts.geometry.y
pts["lon"] = pts.geometry.x

# 6) стартова карта
center = [pts["lat"].median(), pts["lon"].median()] if len(pts) else [48.5, 31.2]
m = folium.Map(location=center, zoom_start=6, tiles="CartoDB positron")

# 7) теплокарта помилок (кожна помилка вагою 1)
heat_data = pts[["lat","lon"]].values.tolist()
HeatMap(heat_data, radius=10, blur=15, min_opacity=0.3).add_to(m)

# 8) опц.: додати точки з найбільшою P(high) серед усього тесту
top = gdf_wgs.sort_values("prob_high", ascending=False).head(2000)
for _, r in top.iterrows():
    folium.CircleMarker(
        location=[r.geometry.y, r.geometry.x],
        radius=3, opacity=0.6, fill=True, fill_opacity=0.6,
        popup=f"y_true={r.y_true}, y_pred={r.y_pred}, Phigh={r.prob_high:.2f}"
    ).add_to(m)

m.save("error_heatmap.html")
print("Saved error_heatmap.html")


Saved error_heatmap.html


In [41]:
import pandas as pd

df = pd.read_parquet("artifacts_fabdem_cv/cv_test_predictions.parquet")
print(df.columns.tolist())
print(df.head())

['x', 'y', 'rgt', 'track', 'spot', 'y_true', 'y_pred', 'prob_low', 'prob_mid', 'prob_high', 'fold']
           x          y    rgt  track  spot  y_true  y_pred  prob_low  \
0  25.061067  47.736959  396.0    3.0   5.0       0       0  0.454448   
1  25.061067  47.736959  396.0    3.0   5.0       0       0  0.454448   
2  25.061066  47.736966  396.0    3.0   5.0       0       0  0.454448   
3  25.061066  47.736966  396.0    3.0   5.0       0       0  0.454448   
4  25.061054  47.737061  396.0    3.0   5.0       0       0  0.427440   

   prob_mid  prob_high  fold  
0  0.353362   0.192189     1  
1  0.353362   0.192189     1  
2  0.353362   0.192189     1  
3  0.353362   0.192189     1  
4  0.327311   0.245249     1  


In [11]:
import pandas as pd
import numpy as np

df = pd.read_parquet("artifacts_fabdem_cv/cv_test_predictions.parquet")

df["model_error"] = (df["y_true"] != df["y_pred"]).astype(int)
df["uncertainty"] = 1.0 - df[["prob_low","prob_mid","prob_high"]].max(axis=1)

df.to_parquet("artifacts_fabdem_cv/cv_test_predictions_with_flags.parquet", index=False)


In [12]:
import numpy as np
import folium
from folium.plugins import HeatMap

df = pd.read_parquet("artifacts_fabdem_cv/cv_test_predictions_with_flags.parquet")

# конвертація 3857 -> 4326


df["lon"] = lon
df["lat"] = lat

# тільки помилки
err = df[df["model_error"] == 1][["lat","lon"]].dropna()

m = folium.Map(location=[df["lat"].median(), df["lon"].median()],
               zoom_start=10, tiles="CartoDB positron")

HeatMap(err.values.tolist(), radius=12, blur=18, max_zoom=12).add_to(m)
m.save("fabdem_errors_heatmap.html")
print("Saved -> fabdem_errors_heatmap.html")



Saved -> fabdem_errors_heatmap.html


In [13]:
unc = df.sort_values("uncertainty", ascending=False).head(20000)  # топ-20k
m2 = folium.Map(location=[df["lat"].median(), df["lon"].median()],
                zoom_start=10, tiles="CartoDB positron")
HeatMap(unc[["lat","lon","uncertainty"]].values.tolist(), radius=13, blur=20).add_to(m2)
m2.save("fabdem_uncertainty_heatmap.html")


In [14]:
hi = df[(df["y_pred"] == 2) & (df["prob_high"] >= 0.8)]
m3 = folium.Map(location=[df["lat"].median(), df["lon"].median()],
                zoom_start=10, tiles="CartoDB positron")
HeatMap(hi[["lat","lon","prob_high"]].values.tolist(), radius=10, blur=16).add_to(m3)
m3.save("fabdem_high_class_hotspots.html")

In [16]:
# приклади категоріальних оглядів
for col in ["lulc_name", "fab_dem_landform", "fab_dem_geomorphon"]:
    if col in df.columns:
        top = (df.groupby(col)["model_error"]
                 .mean()
                 .sort_values(ascending=False)
                 .head(15))
        print(f"\nTop error-rate by {col}:\n", top)

# по рельєфу/нахилах (числові)
if "fab_dem_slope" in df.columns:
    bins = pd.cut(df["fab_dem_slope"], bins=[0,5,10,15,25,35,90])
    print("\nError-rate by slope bins:\n",
          df.groupby(bins)["model_error"].mean())


In [17]:
df = pd.read_parquet("artifacts_fabdem_cv/cv_test_predictions.parquet")

# 1) Бінарна помилка
df["is_error"] = (df["y_true"] != df["y_pred"]).astype(int)

# 2) Невизначеність (чим ближче до 1 → тим модель менш впевнена)
df["uncertainty"] = 1 - df[["prob_low", "prob_mid", "prob_high"]].max(axis=1)

# 3) High-class hotspot (приклад: якщо y_true == 2, і модель часто плутає)
df["is_high_class"] = ((df["y_true"] == 2) & (df["is_error"] == 1)).astype(int)


In [42]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from pyproj import Transformer
import folium
from folium.plugins import HeatMap

# 1) Дані CV
df = pd.read_parquet("artifacts_fabdem_cv/cv_test_predictions.parquet")
gdf3857 = gpd.GeoDataFrame(
    df.copy(),
    geometry=gpd.points_from_xy(df["x"], df["y"]),
    crs="EPSG:4326"     # ВАЖЛИВО: явно вказати
)

# 2) Семплінг HAND (растр у 32635). Координати перетворюємо лише для семплінгу
def sample_raster_to_points(gdf_src, raster_path, out_col):
    with rasterio.open(raster_path) as src:
        rast_crs = src.crs
        nodata = src.nodata
        tr = Transformer.from_crs(gdf_src.crs, rast_crs, always_xy=True)

        xs = gdf_src.geometry.x.values
        ys = gdf_src.geometry.y.values
        xr, yr = tr.transform(xs, ys)     # => координати в CRS растру

        vals = np.array(list(src.sample(zip(xr, yr))))  # (N, bands)
        if vals.ndim == 2 and vals.shape[1] == 1:
            vals = vals[:, 0]
        if nodata is not None:
            vals = np.where(np.isclose(vals, nodata), np.nan, vals)

    gdf_out = gdf_src.copy()
    gdf_out[out_col] = vals
    return gdf_out

gdf3857 = sample_raster_to_points(
    gdf3857, "data/fab_dem_utm32635_hand_2000.tif", out_col="hand"
)

# 3) Підготовка ваг/фільтрів
gdf3857["is_error"] = (gdf3857["y_true"] != gdf3857["y_pred"]).astype(int)
gdf3857["uncertainty"] = 1 - gdf3857[["prob_low","prob_mid","prob_high"]].max(axis=1)

# 4) ПЕРЕД МАПОЮ: проектуємо в WGS84!
gdf4326 = gdf3857.to_crs(4326)

def save_heatmap(gdf, out_html, value_col=None, radius=14, blur=22):
    m = folium.Map(
        location=[gdf.geometry.y.mean(), gdf.geometry.x.mean()],
        zoom_start=10, tiles="cartodbpositron"
    )
    if value_col is None:
        data = list(zip(gdf.geometry.y, gdf.geometry.x))
    else:
        data = list(zip(gdf.geometry.y, gdf.geometry.x, gdf[value_col].fillna(0.0)))
    HeatMap(data, radius=radius, blur=blur, max_zoom=13).add_to(m)
    m.save(out_html)

# 5) Приклади карт
#   а) невизначеність моделі
save_heatmap(gdf4326, "fabdem_uncertainty_heatmap.html", value_col="uncertainty")

#   б) тільки помилки
save_heatmap(gdf4326[gdf4326["is_error"] == 1], "fabdem_errors_heatmap.html")

#   в) помилки в зонах HAND < 5 м (як приклад)
save_heatmap(gdf4326[(gdf4326["is_error"] == 1) & (gdf4326["hand"] < 5)],
             "fabdem_errors_floodplain.html")
